In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Train

In [ ]:
from __future__ import print_function, division

import torch
import torch.utils as utils
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from tqdm import tqdm
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
data_dir = '/content/drive/MyDrive/K-Fashion/'
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_dataset =  datasets.ImageFolder(data_dir, data_transform)

In [ ]:
train_split = 0.7
split_size = int(len(image_dataset) * train_split)
batch_size = 16
num_workers= 6

In [ ]:
train_set, valid_set = torch.utils.data.random_split(image_dataset, [split_size, len(image_dataset) - split_size])
tr_loader = utils.data.DataLoader(dataset=train_set,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=num_workers)
val_loader = utils.data.DataLoader(dataset=valid_set,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=num_workers)
dataloaders = {'train': tr_loader, 'val':val_loader}
dataset_sizes = {}
dataset_sizes['train'] = split_size
dataset_sizes['val'] = len(image_dataset) -split_size
class_names = image_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(image_dataset.classes))
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 291MB/s]


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 2.6817 Acc: 0.2043
val Loss: 2.6571 Acc: 0.2237

Epoch 1/19
----------
train Loss: 2.5517 Acc: 0.2402
val Loss: 2.6154 Acc: 0.2140

Epoch 2/19
----------
train Loss: 2.4512 Acc: 0.2590
val Loss: 2.5609 Acc: 0.2459

Epoch 3/19
----------
train Loss: 2.3596 Acc: 0.2994
val Loss: 2.5536 Acc: 0.2551

Epoch 4/19
----------
train Loss: 2.2891 Acc: 0.3110
val Loss: 2.6832 Acc: 0.2425

Epoch 5/19
----------
train Loss: 2.2261 Acc: 0.3333
val Loss: 2.5584 Acc: 0.2647

Epoch 6/19
----------
train Loss: 2.1496 Acc: 0.3447


##TEST

In [ ]:
class TestDataset(utils.data.Dataset):
  def __init__(self, root, transform=None):
    self.root = root
    self.image_list = os.listdir(root)
    self.transform = transform

  def __len__(self):
    return len(self.image_list)

  def __getitem__(self, index):
    image_path = os.path.join(self.root, self.image_list[index])
    image = np.array(Image.open(image_path))
    image = self.transform(image)
    return self.image_list[index], image

In [ ]:
test_transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/캐주얼/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/캐주얼_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/로맨틱/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/로맨틱_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/유니크/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/유니크_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/심플베이직/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/심플베이직_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/러블리/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/러블리_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/캠퍼스룩/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/캠퍼스룩_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/오피스룩/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/오피스룩_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/빈티지/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/빈티지_resived.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/모던시크/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/모던시크_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/컨템포러리/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/컨템포러리_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/클래식/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/클래식_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/유니섹스/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/유니섹스_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/럭셔리/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/럭셔리_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/섹시글램/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/섹시글램_revised.csv')

In [ ]:
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/아메카지/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
result=[]
for i in range(len(mall)):
  path2 = path + mall[i]
  test_dataset = TestDataset(path2,transform=test_transform)
  test_dataloader = utils.data.DataLoader(test_dataset, batch_size=16, num_workers=16)
  for fnames, data in tqdm(test_dataloader):
    data = data.to(device)
    output = model_ft(data)
    _,pred = torch.max(output,1)
    for j in range(len(fnames)):
      result.append(
          {'filename':fnames[j], 'mall':mall[i], 'style':pred.cpu().detach().numpy()[j]})

df=pd.DataFrame(sorted(result,key=lambda x:x['filename']))
sorted_df = df.sort_values(by='mall')
sorted_df2 = sorted_df.reset_index(drop=True)
sorted_df2.to_csv('/content/drive/MyDrive/데청캠_공유/쇼핑몰 라벨/아메카지_revised.csv')